In [ ]:
# import necessary packages
import openai
import os
import pandas as pd
import dotenv
import base64
import time
import sys
# load open-api key
dotenv.load_dotenv()

# define utility functions
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
# define chat completion function
client = openai.OpenAI()

# define chat completion function
def complete_chat(prompt, base64_image):
    global client
    try:
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                        "type": "text",
                        "text": prompt,
                        },
                        {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "low"
                        }
                        }
                    ],
                }
            ],
            temperature=0.7,
            max_tokens=1,
        )
        response_content = response.choices[0].message.content
        return (int(response_content), None) if response_content.isdigit() and 1 <= int(response_content) <= 7 else (None, None)
    except Exception as e:
        print(f"Error: {e}")
        print(sys.exc_info())
        return True, e
    
# Define a function to process a single image
def process_image(text_prompt, base64_image, image_race, asking_emotion, image_emotion, rater_race):
    response, e = complete_chat(text_prompt, base64_image)
    return {"image_race": image_race, "asking_emotion": asking_emotion, "image_emotion": image_emotion,"rater_race": rater_race, "response": response}, e

In [ ]:
# define prompt template and api completion parameters
prompt="""Imagine you are a{race} person. Your life experiences and background have been largely shaped by your being{race}. As a{race} person you feel your perception of yourself and others is unique. Today you have signed up for a survey to help researchers understand how people perceive emotions in images. 

Researcher: Please look at the image provided. We are interested in understanding how well this image represents the emotion of {emotion}. Using the scale provided below, please indicate your level of agreement with the following statement:
"This image accurately represents the emotion of {emotion}."

1. Strongly Disagree
2. Disagree
3. Somewhat Disagree
4. Neutral
5. Somewhat Agree
6. Agree
7. Strongly Agree

Please select the number that best reflects your perception of the image. Respond with the number rating only and nothing else.

{race} Person:
"""

In [ ]:

results = []
try:
        df = pd.read_csv("rawdata/results.csv").fillna("")
        results = df.to_dict(orient="records")
        done = df.groupby(["image_race", "asking_emotion", "rater_race", "image_emotion"]).count().reset_index()
except:
       done = pd.DataFrame(columns=["image_race", "asking_emotion", "rater_race", "image_emotion", "response"])
       pass

# define list of rater races
rater_races = [" white", " black", ""]

# define rate limit tracking variables
tokens = 0
time_start = time.time()
# define list of images
images = os.listdir("images")

for image in images:
    for rater_race in rater_races:
        # getting data necessary for gathering
        image_emotion, asking_emotion, image_race = image.split(".")[0].split("_")
        image_path = f"images/{image}"
        base64_image = encode_image_to_base64(image_path)
        text_prompt = prompt.format(race=rater_race, emotion=asking_emotion)
        # checking if image has been rated
        curdone = done[(done["image_race"] == image_race) & (done["asking_emotion"] == asking_emotion) & (done["rater_race"] == rater_race) & (done["image_emotion"] == image_emotion)].reset_index(drop=True)
        if len(curdone) > 0:
                curdone = curdone.loc[0, "response"]
                num_trials = 100 - curdone
        else:
                num_trials = 100
        print("\n\n")
        print(f"Num Trials: {num_trials} left for asking emotion {asking_emotion}, image emotion {image_emotion}, image race {image_race}, and Rater Race {rater_race}. Image path is {image_path}")
        print("\n\n")
        #print(text_prompt)
        # running trials
        for trial_num in range(num_trials):
            # making sure responses are valid
                valid = False
                while not valid:
                        # check rate limit
                        if tokens + 289 > 10000:
                                time_end = time.time()
                                time_elapsed = time_end - time_start
                                if time_elapsed < 60:
                                        time.sleep(60 - time_elapsed)
                                tokens = 0
                                time_start = time.time()
                        
                        # fetch response
                        response, e = process_image(text_prompt, base64_image, image_race, asking_emotion, image_emotion, rater_race)
                        tokens += 289
                        print(response)
                        # validating response
                        if (response["response"] is not None and response["response"] != True) or response["response"] == 1:
                                valid = True
                                results.append(response)
                                tokens += 289
                        elif response["response"] == True:
                                print(f"Error: {e}")
                                sys.exit(5)
                
                # checkpoint df
                df = pd.DataFrame(results)
                df.to_csv("rawdata/ckpt.csv", index=False)




In [ ]:
# save results to csv with double check for fallback
df = pd.read_csv("rawdata/ckpt.csv").fillna("")
try:
        old_df = pd.read_csv("rawdata/results.csv").fillna("")
except:
        old_df = pd.DataFrame(columns=["image_race", "asking_emotion", "image_emotion", "rater_race", "response"])
print("New DF")
display(df.groupby(["image_race", "asking_emotion", "image_emotion", "rater_race"]).count().reset_index())
print("Old DF")
display(old_df.groupby(["image_race", "asking_emotion", "image_emotion", "rater_race"]).count().reset_index())
if input("Would you like to overwrite: ") == "y":
        df.to_csv("rawdata/results.csv", index=False)
        print("Overwritten")
        os.remove("rawdata/ckpt.csv")

# Reformatting data for Analysis

In [ ]:
df = pd.read_csv("rawdata/results.csv").fillna("")
analysis_data = pd.DataFrame()
# converting races to dummies 
analysis_data["is_image_white"] = df["image_race"].apply(lambda x: int(x == "W"))
analysis_data["is_image_black"] = df["image_race"].apply(lambda x: int(x == "B"))
analysis_data["is_rater_white"] = df["rater_race"].apply(lambda x: int(x == " white"))
analysis_data["is_rater_black"] = df["rater_race"].apply(lambda x: int(x == " black"))
analysis_data["is_rater_default"] = df["rater_race"].apply(lambda x: int(x == ""))
analysis_data["other_race"] = analysis_data.apply(lambda x: 1 if x["is_image_white"] != x["is_rater_white"] and x["is_rater_default"] == 0 else 0, axis=1)
analysis_data["emotion_congruence"] = df["asking_emotion"] == df["image_emotion"]
analysis_data["emotion_congruence"] = analysis_data["emotion_congruence"].apply(lambda x: int(x))

# adding response
analysis_data["rating"] = df["response"]

# adding emotion for potential categorical analysis
analysis_data["image_emotion"] = df["image_emotion"]

# define variable labels
variable_labels = {
    "is_image_white": "Indicator if image is of a white person",
    "is_image_black": "Indicator if image is of a black person",
    "is_rater_white": "Indicator if rater is white",
    "is_rater_black": "Indicator if rater is black",
    "is_rater_default": "Indicator if rater race is not specified",
    "other_race": "Indicator if image and rater races differ, 0 if rater is default",
    "emotion_congruence": "Indicator of same emotion between asking and image, 1 if same",
    "rating": "Rater's response",
    "image_emotion": "Emotion displayed in the image"
}

# save data for stata
analysis_data.to_stata("rawdata/analysis_data.dta", variable_labels=variable_labels, write_index=False)

# display data for curiousity
display(analysis_data)